https://adventofcode.com/2022/day/11

In [1]:
import re
import operator
from dataclasses import dataclass
from functools import partial, reduce
from math import floor, lcm
from copy import deepcopy
from typing import Callable


# just taking a shortcut here for security reasons 😅
operationMap = {
    "new = old * 19": lambda x: x * 19,
    "new = old + 6": lambda x: x + 6,
    "new = old * old": lambda x: x * x,
    "new = old + 3": lambda x: x + 3,
    "new = old * 7": lambda x: x * 7,
    "new = old * 13": lambda x: x * 13,
    "new = old + 1": lambda x: x + 1,
    "new = old + 7": lambda x: x + 7,
    "new = old + 4": lambda x: x + 4,
    "new = old + 8": lambda x: x + 8,
}


@dataclass
class Monkey:
    items: list[int]
    operation: Callable[[int], int]
    test: Callable[[int], int]
    counter: int = 0


def parseMonkeys(input_file: str) -> list[Monkey]:
    with open(input_file) as f:
        text = f.read()

    monkeys = list()
    testTemplate = lambda x, div, mTrue, mFalse: mTrue if not (x % div) else mFalse
    for item in text.split("\n\n"):
        lines = item.splitlines()
        items = list(map(lambda x: int(x.strip()), lines[1].split(": ")[1].split(",")))
        operation = operationMap[lines[2].split(":")[1].strip()]
        div, mTrue, mFalse = map(lambda l: int(re.search(r"(\d+)", l)[0]), lines[3:])
        test = partial(testTemplate, div=div, mTrue=mTrue, mFalse=mFalse)
        monkeys.append(Monkey(items=items, operation=operation, test=test))

    return monkeys


def solvePart1(monkeys: list[Monkey]) -> int:
    for _ in range(20):
        for m in monkeys:
            for item in m.items:
                m.counter += 1
                level = m.operation(item)
                level = floor(level / 3)
                monkeys[m.test(level)].items.append(level)
            m.items = list()

    return reduce(operator.mul, sorted([m.counter for m in monkeys], reverse=True)[:2], 1)


def solvePart2(monkeys: list[Monkey]) -> int:
    common = lcm(*[m.test.keywords["div"] for m in monkeys])
    for _ in range(10000):
        for m in monkeys:
            for item in m.items:
                m.counter += 1
                level = m.operation(item)
                monkeys[m.test(level)].items.append(level % common)
            m.items = list()

    return reduce(operator.mul, sorted([m.counter for m in monkeys], reverse=True)[:2], 1)


In [2]:
monkeys = parseMonkeys("test_input.txt")

result, expected = solvePart1(deepcopy(monkeys)), 10605
assert result == expected, f"Part 1: {result=} is wrong ({expected=})"

result, expected = solvePart2(deepcopy(monkeys)), 2713310158
assert result == expected, f"Part 2: {result=} is wrong ({expected=})"

In [3]:
monkeys = parseMonkeys("input.txt")

print(solvePart1(deepcopy(monkeys)))
print(solvePart2(deepcopy(monkeys)))

78960
14561971968
